In [5]:
import os, sys
import numpy as np
import pandas as pd

import matplotlib
%matplotlib inline
#matplotlib.use('Agg')
import matplotlib.pyplot as plt

plt.style.use('ggplot')

caffe_root = '/home/ashu/Desktop/Thesis Work/Classifier/caffe/'
print os.getcwd()
sys.path.insert(0, caffe_root + 'python')
import caffe

from PIL import Image
import argparse

from caffe.proto import caffe_pb2
import google.protobuf as pb2
import google.protobuf.text_format

import lmdb
from collections import defaultdict

/home/ashu/Desktop/Thesis Work/Classifier/caffe/models/script


- Load Detections of Val Set for the category specified

In [16]:
object_name = 'umbrella'
det_file = caffe_root + 'data/det_val/det_val_' + object_name + '.txt'

with open(det_file, 'r') as f:
    lines = np.array(f.readlines()) 
    
remove_new_line = np.vectorize(lambda x: x.strip())
lines = remove_new_line(lines)

print len(lines)
print lines[:7]

images_root = '/home/ashu/Study Material/Uni Bonn/4th Semester/Thesis/DataSet/coco-master/JPEGImages/'
#temporary images folder to save cropped images
tmp_img = caffe_root + 'models/gt_classifier/tmp_images/' + object_name + '/'

18966
['COCO_val2014_000000568337 0.269 383.0 183.7 470.4 222.1'
 'COCO_val2014_000000568337 0.162 38.2 84.8 383.2 428.0'
 'COCO_val2014_000000193720 0.551 327.6 90.4 543.0 422.4'
 'COCO_val2014_000000039663 0.299 331.4 60.1 347.8 149.0'
 'COCO_val2014_000000039663 0.268 374.5 71.7 390.2 144.3'
 'COCO_val2014_000000039663 0.099 321.9 57.8 337.6 145.6'
 'COCO_val2014_000000504224 0.444 391.6 395.8 634.4 480.0']


- Load Images, crop the detections and save it into a temporary folder

In [138]:

if not os.path.exists(tmp_img):
    os.makedirs(tmp_img)

for i, det in enumerate(lines):  # for first 10 only
    val = det.split(' ')
    img = Image.open(images_root + val[0] + '.jpg')
    #print val[0]
    new_img = img.crop((float(val[2]), float(val[3]), float(val[4]), float(val[5]))) #.resize((224,224))
    new_img_name = val[0] + '_' + str(i) + '.jpg'
    new_img.save(tmp_img + new_img_name)
    

- Open two files for open and close State
- Loop Through list, load image, crop ,resize and feed into classifier
- predict state labels, fc7 feature and confidence
- According to labels add Img Id, confidence, x1, y1, x2, y2 to open or close state file
- add fc7 feature and label to features file

In [17]:
# list for storing detections of each state
open_state = []
close_state = []
fc7_vectors = []
label_list = []

# tmp saved image path
tmp_img = caffe_root + 'models/gt_classifier/tmp_images/' + object_name + '/'

caffe.set_mode_gpu()

max_iter = '40000'
best_iter = '30000'
# Modify the paths given below
deploy_prototxt_file_path = caffe_root + 'models/gt_classifier/gen_' + object_name +  \
                            '/test.prototxt' # Network definition file
# load each caffe model
caffe_model_file_path = caffe_root + 'models/gt_classifier/gen_' + object_name + \
                        '/gen_' + object_name + '_' + max_iter + \
                        '_iter_' + best_iter + '.caffemodel' 
        
net = caffe.Net(deploy_prototxt_file_path,      # defines the structure of the model
                caffe_model_file_path,  # contains the trained weights
                caffe.TEST)     # use test mode (e.g., don't perform dropout)

# load the mean ImageNet image (as distributed with Caffe) for subtraction
mu = np.load(caffe_root + 'python/caffe/imagenet/ilsvrc_2012_mean.npy')
mu = mu.mean(1).mean(1)  # average over pixels to obtain the mean (BGR) pixel values
print 'mean-subtracted values:', zip('BGR', mu)

# create transformer for the input called 'data'
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})

transformer.set_transpose('data', (2,0,1))  # move image channels to outermost dimension
transformer.set_mean('data', mu)            # subtract the dataset-mean value in each channel
transformer.set_raw_scale('data', 255)      # rescale from [0, 1] to [0, 255]
transformer.set_channel_swap('data', (2,1,0))  # swap channels from RGB to BGR

net.blobs['data'].reshape(1,        # batch size
                          3,         # 3-channel (BGR) images
                          224, 224)  # image size is 227x227

for i, det in enumerate(lines):  # for first 10 only
    val = det.split(' ')
    new_img_name = val[0] + '_' + str(i) + '.jpg'
    # copy the image data into the memory allocated for the net
    image = caffe.io.load_image(tmp_img + new_img_name)
    transformed_image = transformer.preprocess('data', image)
    net.blobs['data'].data[...] = transformed_image

    ### perform classification
    output = net.forward()

    output_prob = output['prob']  # the output probability vector for the first image in the batch

    #print 'predicted score: ', output_prob
    state_label = output_prob[0].argmax()
    #print i, ': predicted class is:', state_label
    
    cnf = output_prob[0][output_prob.argmax()]
    
    # confidence = cnf by detections * cnf of classifier
    new_cnf = cnf * float(val[1])
    new_det = val[0] + ' ' + str(new_cnf) + ' ' + val[2] + ' ' + val[3] + ' ' + val[4] + ' ' + val[5]
    
    # append fc7 vectir and labels to list
    fc7_vectors.append(net.blobs['fc7'].data.copy())
    label_list.append(state_label)
    
    if state_label == 0:
        open_state.append(new_det + '\n')
    else:
        close_state.append(new_det + '\n')
        
print 'Open: \n ', open_state[:2]
print 'Close: \n ', close_state[:2]
#print 'fc7: \n', fc7_vectors[-4:]


mean-subtracted values: [('B', 104.0069879317889), ('G', 116.66876761696767), ('R', 122.6789143406786)]
Open: 
  ['COCO_val2014_000000568337 0.267875399828 383.0 183.7 470.4 222.1\n', 'COCO_val2014_000000568337 0.161996214867 38.2 84.8 383.2 428.0\n']
Close: 
  ['COCO_val2014_000000193720 0.301764433086 327.6 90.4 543.0 422.4\n', 'COCO_val2014_000000039663 0.298997968316 331.4 60.1 347.8 149.0\n']


In [24]:
# chek fc7 output
fc7 = np.array(fc7_vectors)
print fc7.shape[0]
for i in range(fc7.shape[0]):
    if not np.allclose(fc7[0][0], fc7[i][0]):
        print i
        break
       

4812
1


- Save Detections in seperate folder for open and close state classified acc to classifier

In [18]:
det_dir = caffe_root + 'models/gt_classifier/gen_' + object_name + '/detections/'
if not os.path.exists(det_dir):
    os.makedirs(det_dir)
    
open_fileName = det_dir + 'gen_val_open'  + object_name.title() + '.txt'
close_fileName = det_dir + 'gen_val_closed'  + object_name.title() + '.txt'

print open_fileName, close_fileName

#write into file
with open(open_fileName, 'w+') as f:
    for val in open_state:
        f.write(val)
        
#write into file
with open(close_fileName, 'w+') as f:
    for val in close_state:
        f.write(val)


/home/ashu/Desktop/Thesis Work/Classifier/caffe/models/gt_classifier/gen_umbrella/detections/gen_val_openUmbrella.txt /home/ashu/Desktop/Thesis Work/Classifier/caffe/models/gt_classifier/gen_umbrella/detections/gen_val_closedUmbrella.txt


- Save fc7 feature and its label using pickle dump

In [39]:
import pickle

det_dir = caffe_root + 'models/gt_classifier/' + object_name + '/detections/'

fc7_file = det_dir+ 'gt_val_fc7_' + object_name + '.pkl'
lbl_file = det_dir+ 'gt_val_fc7_label_' + object_name + '.pkl'

'''
def format(value):
    return "%.3f" % value

formatted = [[format(v) for v in r] for r in fc7_vectors]
'''
# dump fc7 features
fileObject = open(fc7_file,'wb')
pickle.dump(fc7_vectors,fileObject)
fileObject.close()

# dump state labels
fileObject = open(lbl_file,'wb')
pickle.dump(label_list,fileObject)
fileObject.close()

print len(fc7_vectors), len(label_list)
print fc7_vectors[:3]
print label_list[:3]

4265 4265
[array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.17291714]], dtype=float32), array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.70667338]], dtype=float32), array([[ 0.        ,  0.        ,  0.        , ...,  0.08704553,
         0.06781743,  1.52062988]], dtype=float32)]
[1, 1, 1]


- load pickle and check

In [6]:
fileObject = open(fc7_file,'r')  
# load the object from the file into var b
loaded_fc7 = pickle.load(fileObject)  
fileObject.close()

fileObject = open(lbl_file,'r')  
# load the object from the file into var b
loaded_lbl= pickle.load(fileObject)  
fileObject.close()

print len(loaded_fc7), len(loaded_lbl)
print loaded_fc7[:3]
print loaded_lbl[:3]

8942 8942
[array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  2.33277202]], dtype=float32), array([[ 0.        ,  0.32808748,  0.        , ...,  0.        ,
         1.74012995,  2.361202  ]], dtype=float32), array([[ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.5555228]], dtype=float32)]
[0, 0, 0]
